In [136]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import os
import re
import numpy as np
from copy import deepcopy
import random
import scipy.sparse as sp

# directory = '../MF/ml-1m'
# ratings = []
# with open(os.path.join(directory, 'ratings.dat'), encoding='latin1') as f:
#     for l in f:
#         user_id, movie_id, rating, timestamp = [int(_) for _ in l.split('::')]
#         ratings.append({
#             'user_id': user_id,
#             'movie_id': movie_id,
#             'rating': rating,
#             'timestamp': timestamp,
#             })
# ratings = pd.DataFrame(ratings)

# ratings = ratings.drop(['timestamp'],axis=1)
# print(ratings)
# # 어떤 비율로 표본 추출을 하고싶은지 df.sample에서 frac을 0~1 사이로 설정
# df = ratings.sample(frac=1).reset_index(drop=True)
# filter_user = deepcopy(df)
# counts = filter_user['user_id'].value_counts()
# filter_user = filter_user[filter_user['user_id'].isin(counts[counts >= 10].index)]

# filtered_df = deepcopy(filter_user)
# counts = filtered_df['movie_id'].value_counts()
# filtered_df = filtered_df[filtered_df['movie_id'].isin(counts[counts >=10].index)]
# filtered_df = filtered_df.reset_index(drop=True)
# print(filtered_df)


neighbor_dict = {}
user = []
item = []

with open('../NGCF-PyTorch/Data/ml-1m/train.txt', 'r') as f:
    for l in f.readlines():
        if len(l) > 0:
            l = l.strip('\n').split(' ')
            neighbor_dict[int(l[0])] = [int(i) for i in l[1:]]
            user.append(int(l[0]))

def jaccard_index(u_i, u_j, neighbor_dict):
    u_i_neighbor = neighbor_dict[u_i]
    u_j_neighbor = neighbor_dict[u_j]
    return len(list(set(u_i_neighbor) & set(u_j_neighbor))) / len(list(set(u_i_neighbor) | set(u_j_neighbor)))

# 새로운 graph의 node j 에다가 기존 graph의 어떤 node의 neighborhood를 복사할지 zeta에 담기
def node_copying(user):
    zeta = []
    for u_j in user:
        nor = 0
        zeta_distribution = []
        for u_i in user:
            nor+=jaccard_index(u_j, u_i, neighbor_dict)
        for u_m in user:
            zeta_distribution.append(jaccard_index(u_j, u_m, neighbor_dict) / nor)
        zeta.append(random.choices(user, weights=zeta_distribution)[0])
    return zeta

# zeta = node_copying(user)

[0, 1, 2, 3, 7, 0, 6, 8, 8, 9]

In [135]:
# node_copying 함수로 zeta를 구한 후 인자로 사용
# epoch만큼 graph generate
def generate_graph(user, zeta, neighbor_dict, epsilon):
    generated_node = []
    sampled_graph = dict()
    for i in user:
        if random.uniform(0,1) < 1-epsilon:  # 1-epsilon의 확률로 원래 neighbor 넣기
            generated_node.append(i)
        else:                                # epsilon의 확률로 zeta에 있는 node의 neighbor로 copy해서 넣기
            generated_node.append(zeta[i])
        sampled_graph[i] = neighbor_dict[generated_node[i]]
    return sampled_graph
    
# sampled_graph = generate_graph(user, zeta, neighbor_dict, 0.2)

In [ ]:
# GAT
# 만들어지는 graph마다 embedding 초기화 
# embedding 학습하는 GNN 구조 짜기
# 학습된 embedding 가지고 x_hat (eq.11) 구하고 BPR-OPT를 maximization 시키는 방향으로 학습

In [137]:
class AttenConv(nn.Module):
    def __init__(self):
        super(AttenConv, self).__init__()
    
    def forward(self):
        pass

In [202]:
# data로 sparse matrix 및 adjacency matrix 만들기

class Data(object):
    def __init__(self, path):
        self.path =path
        
        train_file = path + '/trian.txt'
        # path : ../NGCF-PyTorch/Data/ml-1m
        
        self.n_users, self.n_items = 0, 0
        self.exist_users = []
        with open(train_file, 'r') as f:
            for l in f.readlines():
                if len(l) > 0:
                    l = l.strip('\n').split(' ')
                    items = [int(i) for i in l[1:]]
                    uid = int(l[0])
                    self.exist_users.append(uid)
                    self.n_users = max(n_users, uid)
                    self.n_items = max(n_items, max(items))

        self.n_users+=1
        self.n_items+=1
        print(n_users, n_items)

        self.R = sp.dok_matrix((self.n_users, self.n_items), dtype=np.float32)
        self.train_items = {}

        with open(train_file, 'r') as f:
            for l in f.readlines():
                if len(l)==0:
                    break
                l = l.strip('\n').split(' ')
                uid, items = int(l[0]), [int(i) for i in l[1:]]

                for i in items:
                    self.R[uid, i] = 1

                self.train_items[uid] = items
                
    def get_adj_mat(self):
        try:
            adj_mat = sp.load_npz(self.path + '/s_adj_mat.npz')
            
        except Exception:
            adj_mat = self.create_adj_mat()
            sp.save_npz(self.path + '/s_adj_mat.npz', adj_mat)
            
        return adj_mat
    
    
    def create_adj_mat(self):
        adj_mat = self.R.todok()
        print('already create adjacency matrix', adj_mat.shape)
        return adj_mat.tocsr()


6040 3953
  (0, 720)	1.0
  (0, 1836)	1.0
  (0, 3186)	1.0
  (0, 2028)	1.0
  (0, 745)	1.0
  (0, 3114)	1.0
  (0, 595)	1.0
  (0, 1035)	1.0
  (0, 3105)	1.0
  (0, 914)	1.0
  (0, 783)	1.0
  (0, 1028)	1.0
  (0, 1545)	1.0
  (0, 527)	1.0
  (0, 1)	1.0
  (0, 2398)	1.0
  (0, 1907)	1.0
  (0, 1207)	1.0
  (0, 2340)	1.0
  (0, 1962)	1.0
  (0, 594)	1.0
  (0, 1029)	1.0
  (0, 1197)	1.0
  (0, 2687)	1.0
  (0, 2018)	1.0
  :	:
  (6039, 30)	1.0
  (6039, 1641)	1.0
  (6039, 1219)	1.0
  (6039, 2349)	1.0
  (6039, 1449)	1.0
  (6039, 592)	1.0
  (6039, 1127)	1.0
  (6039, 1150)	1.0
  (6039, 17)	1.0
  (6039, 3224)	1.0
  (6039, 3168)	1.0
  (6039, 265)	1.0
  (6039, 2022)	1.0
  (6039, 296)	1.0
  (6039, 2872)	1.0
  (6039, 1212)	1.0
  (6039, 1419)	1.0
  (6039, 2692)	1.0
  (6039, 2407)	1.0
  (6039, 1249)	1.0
  (6039, 2391)	1.0
  (6039, 25)	1.0
  (6039, 3068)	1.0
  (6039, 910)	1.0
  (6039, 1193)	1.0


In [193]:
a = np.matrix([[0,1,2],[1,1,1],[0,0,1]])
adj = np.matrix([[0,0,1],[1,0,0],[1,1,0]])
e = np.matrix([[1,2,3],[4,5,6],[7,8,9]])

print(a,'\n')
print(adj,'\n')
print(e,'\n')

a_jk = np.multiply(a,adj)
print(a_jk, '\n')

np.matmul(a_jk, e)
    
    
    

[[0 1 2]
 [1 1 1]
 [0 0 1]] 

[[0 0 1]
 [1 0 0]
 [1 1 0]] 

[[1 2 3]
 [4 5 6]
 [7 8 9]] 

[[0 0 2]
 [1 0 0]
 [0 0 0]] 



matrix([[14, 16, 18],
        [ 1,  2,  3],
        [ 0,  0,  0]])